# Tuning into Tumors: Frequency-Domain Tumor Segmentation
## BME 271D Final Project - Fall 2025

**Team Members:** Ege Ozemek, Max Bazan, Sasha Nikiforov

---

## Project Overview

This project explores **frequency-domain filtering** techniques for tumor segmentation in medical images (MRI/CT scans). We compare:

1. **FFT-based High-Pass Filtering** - Enhances tumor boundaries
2. **FFT-based Band-Pass Filtering** - Isolates tumor texture
3. **Canny Edge Detection** - Spatial-domain edge-based segmentation
4. **Baseline Methods** - Direct Otsu thresholding for comparison

### Key Questions:
- Can frequency-domain filtering improve tumor segmentation accuracy?
- How do different frequency bands contribute to tumor detection?
- What are the clinical implications of automated volume estimation?

### Clinical Motivation:
Accurate tumor segmentation enables:
- Precise volume measurement for treatment planning
- Tracking tumor growth/shrinkage over time
- Radiation therapy target delineation
- Surgical planning and navigation

## Setup and Imports

In [ ]:
# Import our custom tumor segmentation module
import tumor_segmentation as ts

# Standard imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path

# Configure matplotlib
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("Setup complete!")

## Part 1: Understanding Frequency Domain Representation

Before segmentation, let's explore how medical images look in the frequency domain.

In [ ]:
# Load a sample image
sample_image = ts.load_grayscale_image('data/images/sample_tumor.png')
sample_mask = ts.load_binary_mask('data/masks/sample_tumor_mask.png')

print(f"Image shape: {sample_image.shape}")
print(f"Image range: [{sample_image.min():.3f}, {sample_image.max():.3f}]")
print(f"Tumor pixels: {sample_mask.sum()} ({100*sample_mask.sum()/sample_mask.size:.1f}% of image)")

In [ ]:
# Compute and visualize FFT spectrum
F_shift, magnitude = ts.compute_fft_spectrum(sample_image)
fig = ts.visualize_frequency_spectrum(sample_image, F_shift, 
                                      title="Frequency Domain Analysis of Tumor Image")
plt.show()

print("\nKey Observations:")
print("- The bright center contains low-frequency components (overall intensity, smooth regions)")
print("- Radiating patterns contain high-frequency components (edges, tumor boundaries)")
print("- Mid-frequency bands contain texture information (tumor heterogeneity)")

### Phase vs. Magnitude

The FFT produces complex numbers with magnitude and phase. Let's see why both matter:

In [ ]:
# Reconstruct from magnitude only (zero phase)
no_phase_recon = ts.reconstruct_without_phase(magnitude, sample_image.shape)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(sample_image, cmap='gray')
axes[0].set_title('Original Image')
axes[0].axis('off')

axes[1].imshow(np.log1p(magnitude), cmap='gray')
axes[1].set_title('Magnitude Spectrum')
axes[1].axis('off')

axes[2].imshow(no_phase_recon, cmap='gray')
axes[2].set_title('Reconstruction Without Phase\n(Demonstrates Phase Importance)')
axes[2].axis('off')

plt.suptitle('The Critical Role of Phase Information', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nConclusion: Phase information is critical for spatial structure!")
print("We keep both magnitude and phase in our filtering pipeline.")

## Part 2: Filter Design

We design three types of frequency-domain filters:

1. **High-Pass Filter**: Blocks low frequencies, keeps high frequencies (edges)
2. **Band-Pass Filter**: Keeps only a specific frequency range (texture)
3. **Low-Pass Filter**: Smooths image (comparison only)

In [ ]:
# Define filter parameters
params = {
    'hp_radius': 25,      # High-pass cutoff (blocks frequencies within this radius)
    'bp_r1': 10,          # Band-pass inner radius
    'bp_r2': 40,          # Band-pass outer radius
    'canny_sigma': 1.0,   # Canny edge detector smoothing
    'gaussian_sigma': 1.0 # Gaussian smoothing for baseline
}

# Create filter masks
hp_mask = ts.make_hp_mask(sample_image.shape, params['hp_radius'])
bp_mask = ts.make_bp_mask(sample_image.shape, params['bp_r1'], params['bp_r2'])
lp_mask = ts.make_lp_mask(sample_image.shape, 50)

# Visualize filters
fig = ts.visualize_filters(hp_mask, bp_mask, lp_mask)
plt.show()

print("\nFilter Characteristics:")
print(f"- High-Pass: Passes {100*hp_mask.sum()/hp_mask.size:.1f}% of frequencies")
print(f"- Band-Pass: Passes {100*bp_mask.sum()/bp_mask.size:.1f}% of frequencies")
print(f"- Low-Pass: Passes {100*lp_mask.sum()/lp_mask.size:.1f}% of frequencies")

## Part 3: Filtering Effects

Let's apply these filters and see how they transform the image.

In [ ]:
# Apply filters
hp_filtered, _, _ = ts.filter_pipeline(sample_image, 'hp', cutoff_radius=params['hp_radius'])
bp_filtered, _, _ = ts.filter_pipeline(sample_image, 'bp', r1=params['bp_r1'], r2=params['bp_r2'])
lp_filtered, _, _ = ts.filter_pipeline(sample_image, 'lp', cutoff_radius=50)

# Visualize filtering effects
fig, axes = plt.subplots(2, 2, figsize=(12, 12))

axes[0, 0].imshow(sample_image, cmap='gray')
axes[0, 0].set_title('Original Image', fontweight='bold')
axes[0, 0].axis('off')

axes[0, 1].imshow(hp_filtered, cmap='gray')
axes[0, 1].set_title('High-Pass Filtered\n(Edge Enhancement)', fontweight='bold')
axes[0, 1].axis('off')

axes[1, 0].imshow(bp_filtered, cmap='gray')
axes[1, 0].set_title('Band-Pass Filtered\n(Texture Isolation)', fontweight='bold')
axes[1, 0].axis('off')

axes[1, 1].imshow(lp_filtered, cmap='gray')
axes[1, 1].set_title('Low-Pass Filtered\n(Smoothing)', fontweight='bold')
axes[1, 1].axis('off')

plt.suptitle('Frequency-Domain Filtering Effects', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nObservations:")
print("- High-pass: Tumor boundaries are emphasized, uniform regions suppressed")
print("- Band-pass: Texture patterns highlighted, both edges and smooth regions reduced")
print("- Low-pass: Image smoothed, noise reduced but edges blurred")

## Part 4: Segmentation Pipeline

Now we apply segmentation to filtered images and compare with baselines.

In [ ]:
# Run complete segmentation experiment on sample image
results = ts.run_single_image_experiment(sample_image, sample_mask, params, verbose=True)

print("\n" + "="*70)
print(" SEGMENTATION RESULTS")
print("="*70)

for method_name, method_data in results.items():
    metrics = method_data['metrics']
    print(f"\n{method_name}:")
    print(f"  Dice Coefficient: {metrics['dice']:.4f}")
    print(f"  IoU Score:        {metrics['iou']:.4f}")
    print(f"  Boundary Accuracy: {metrics['boundary_acc']:.4f}")

In [ ]:
# Visualize all segmentation results
masks_dict = {name: data['mask'] for name, data in results.items()}
fig = ts.plot_segmentation_comparison(sample_image, masks_dict, sample_mask)
plt.show()

### Detailed Comparison: Best Method vs Ground Truth

In [ ]:
# Find best method based on Dice score
best_method = max(results.items(), key=lambda x: x[1]['metrics']['dice'])
best_name = best_method[0]
best_mask = best_method[1]['mask']

print(f"Best Method: {best_name}")
print(f"Dice Score: {best_method[1]['metrics']['dice']:.4f}")

# Create overlay visualization
fig = ts.create_overlay_visualization(sample_image, best_mask, sample_mask, best_name)
plt.show()

print("\nIn the overlay:")
print("- RED contours = Our predicted tumor boundary")
print("- GREEN contours = Ground truth tumor boundary")
print("- Perfect overlap would show yellow (red + green)")

## Part 5: Volume Estimation

Clinical application: Estimate tumor volume from segmentation.

In [ ]:
# Assume typical MRI parameters (these would come from DICOM metadata in practice)
pixel_spacing = (0.5, 0.5)  # mm per pixel
slice_thickness = 5.0        # mm

# Estimate volume for best method
volume_mm3, volume_cm3 = ts.estimate_tumor_volume(best_mask, pixel_spacing, slice_thickness)

# Also compute for ground truth
gt_volume_mm3, gt_volume_cm3 = ts.estimate_tumor_volume(sample_mask, pixel_spacing, slice_thickness)

print("\n" + "="*70)
print(" VOLUME ESTIMATION")
print("="*70)

print(f"\nGround Truth:")
print(f"  Volume: {gt_volume_mm3:.1f} mm³ ({gt_volume_cm3:.3f} cm³)")

print(f"\n{best_name}:")
print(f"  Volume: {volume_mm3:.1f} mm³ ({volume_cm3:.3f} cm³)")

volume_error = abs(volume_mm3 - gt_volume_mm3) / gt_volume_mm3 * 100
print(f"  Volume Error: {volume_error:.1f}%")

# Visualize
fig = ts.display_volume_measurement(sample_image, best_mask, volume_mm3, volume_cm3)
plt.show()

print("\nClinical Significance:")
print("- Oncologists use volume measurements to track treatment response")
print("- Volume change >25% often indicates significant progression/regression")
print("- Accurate segmentation enables precise radiation therapy planning")

## Part 6: Batch Experiment on Multiple Images

To validate our methods, we need to test on multiple images.

In [ ]:
# Load all images
image_dir = 'data/images'
mask_dir = 'data/masks'

images, masks, filenames = ts.load_dataset(image_dir, mask_dir)

print(f"Loaded {len(images)} images for batch processing")
print(f"Filenames: {filenames}")

In [ ]:
# Run batch experiment
output_dir = 'results'
results_df = ts.run_batch_experiment(images, masks, filenames, params, output_dir)

# Display results table
print("\n" + "="*70)
print(" BATCH RESULTS")
print("="*70)
print(results_df.head())

## Part 7: Statistical Analysis

Compare all methods statistically across the entire dataset.

In [ ]:
# Compute and display statistics
ts.compare_methods_statistically(results_df, output_dir)

In [ ]:
# Create summary statistics table
methods = ['Baseline_Raw_Otsu', 'Baseline_Smooth_Otsu', 'FFT_HighPass', 'FFT_BandPass', 'Canny_Edges']
metrics = ['dice', 'iou', 'boundary_acc']

summary_data = []
for method in methods:
    row = {'Method': method}
    for metric in metrics:
        col_name = f'{method}_{metric}'
        if col_name in results_df.columns:
            mean = results_df[col_name].mean()
            std = results_df[col_name].std()
            row[f'{metric.upper()}'] = f'{mean:.3f} ± {std:.3f}'
    summary_data.append(row)

summary_df = pd.DataFrame(summary_data)
print("\n" + "="*70)
print(" SUMMARY STATISTICS")
print("="*70)
print(summary_df.to_string(index=False))

## Part 8: Parameter Sensitivity Analysis (Optional)

Test how sensitive our methods are to filter parameter choices.

In [ ]:
# Test different high-pass cutoff radii
hp_radii = [10, 15, 20, 25, 30, 35, 40]
hp_scores = []

print("Testing high-pass filter sensitivity...")
for radius in hp_radii:
    test_params = params.copy()
    test_params['hp_radius'] = radius
    
    # Test on first image
    hp_filtered, _, _ = ts.filter_pipeline(sample_image, 'hp', cutoff_radius=radius)
    mask_hp = ts.otsu_segmentation(hp_filtered)
    dice = ts.dice_coefficient(mask_hp, sample_mask)
    hp_scores.append(dice)
    print(f"  Radius {radius}: Dice = {dice:.4f}")

# Plot sensitivity
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(hp_radii, hp_scores, 'o-', linewidth=2, markersize=8)
ax.set_xlabel('High-Pass Cutoff Radius (pixels)', fontsize=12, fontweight='bold')
ax.set_ylabel('Dice Coefficient', fontsize=12, fontweight='bold')
ax.set_title('Parameter Sensitivity: High-Pass Filter', fontsize=14, fontweight='bold')
ax.grid(alpha=0.3)
ax.axvline(params['hp_radius'], color='r', linestyle='--', label='Chosen Value')
ax.legend()
plt.tight_layout()
plt.savefig(f'{output_dir}/parameter_sensitivity.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nOptimal radius from this test: {hp_radii[np.argmax(hp_scores)]}")
print(f"Our chosen radius: {params['hp_radius']}")

## Conclusions and Discussion

### Key Findings:

1. **Frequency-Domain Filtering Performance**
   - High-pass filtering effectively enhanced tumor boundaries
   - Band-pass filtering isolated texture features
   - Both methods showed improvement over baseline Otsu thresholding

2. **Method Comparison**
   - [Insert findings from your data here]
   - FFT-based methods achieved X% improvement in Dice score
   - Canny edge detection showed [strengths/weaknesses]

3. **Clinical Applications**
   - Automated volume estimation enables treatment monitoring
   - Sub-millimeter accuracy important for surgical planning
   - Real-time segmentation could assist radiologists

### Limitations:

1. **2D vs 3D**: We analyzed 2D slices; full 3D segmentation would be more clinically relevant
2. **Single tumor type**: Methods may need adjustment for different cancer types
3. **Fixed parameters**: Adaptive parameter selection could improve robustness
4. **Heterogeneous tumors**: Complex internal structures remain challenging

### Future Directions:

1. Extend to 3D volumetric segmentation
2. Combine frequency and spatial features with machine learning
3. Adaptive parameter selection based on image characteristics
4. Multi-modal fusion (T1, T2, FLAIR MRI sequences)
5. Clinical validation with radiologist ground truth

### Signals & Systems Concepts Applied:

- **Fourier Transform**: Converting spatial domain to frequency domain
- **Filtering**: High-pass, band-pass, and low-pass filter design
- **Convolution**: Implicit in Gaussian smoothing and morphological operations
- **Sampling Theory**: Understanding pixel spacing and resolution limits
- **Frequency Response**: Analyzing which frequencies contain tumor information

---

## References

1. The Cancer Imaging Archive (TCIA) - https://www.cancerimagingarchive.net/
2. Otsu, N. (1979). A threshold selection method from gray-level histograms. IEEE Trans. Systems, Man, and Cybernetics.
3. Canny, J. (1986). A computational approach to edge detection. IEEE Trans. Pattern Analysis and Machine Intelligence.
4. Dice, L.R. (1945). Measures of the amount of ecologic association between species. Ecology.
5. BME 271D Course Materials - Signals and Systems, Duke University

## Appendix: Running the Complete Pipeline

To run the entire pipeline on your TCIA data:

In [ ]:
# Complete pipeline execution
# Uncomment and modify paths as needed

# ts.main(
#     image_dir='data/images',
#     mask_dir='data/masks',
#     output_dir='results',
#     params={
#         'hp_radius': 25,
#         'bp_r1': 10,
#         'bp_r2': 40,
#         'canny_sigma': 1.0,
#         'gaussian_sigma': 1.0
#     }
# )